In [ ]:
import json
import os
import re
import subprocess
import time

In [ ]:
# NOTE: Until SDL is patched in a future version of scrcpy, run `set SDL_AUDIODRIVER=directsound` before running any of these scripts
# Commands to run scrcpy with Quest 2 crop. directsound probably not needed
# set SDL_AUDIODRIVER=directsound
# scrcpy --video-bit-rate=2M --max-fps=24 --crop=1600:900:2017:510

In [ ]:
# TODO: Reconnect all TCPIP_DEVICES, cycle between current scrcpy procs, error logging, proximity sensor toggle

"""
Flow:
1. User plugs in device via USB
2. Clicks a button to activate TCPIP mode
3. User authenticates on the device if necessary
4. Clicks a button to activate scrcpy, which goes full screen
5. Can click a button to disable proximity sensor for the current device
6. Can click a button to cycle through devices. Add sleep 3 secs to kill to account for initial loading time
7. Can click a button to close scrcpy

Other features:
1. Can click a button to attempt to reconnect to all previously known devices (stored in a .json)
2. Can click a button to disable tcpip on all devices
3. Can click a button to disconnect all TCPIP devices

Stretch:
1. Make it detect USB events automatically and attempt to activate TCPIP on newly connected devices
2. Display multiple devices at once
3. Spoof internet connection on hotspot
""";

In [ ]:
TCPIP_DEVICES = []
CURR_IX = 0
SCRCPY_PROC = None

In [ ]:
def get_devices():
    q = ["adb", "devices"]
    proc = subprocess.Popen(q, stdin=subprocess.PIPE, stdout=subprocess.PIPE)
    stdout, _ = proc.communicate()
    stdout = stdout.decode()

    devices = stdout.split("\n")
    devices = [s for s in devices if "\tdevice" in s]

    devices = [s.split("\t")[0] for s in devices]

    return devices

In [ ]:
def get_connected_devices():
    devices = get_devices()
    return [s for s in devices if re.match(r"\b\d+(?:\.\d+){3}\b", s)]

In [ ]:
def get_usb_devices():
    devices = get_devices()
    return [s for s in devices if s not in get_connected_devices()]

In [ ]:
def get_device_model(device):
    q = ["adb", "-s", device, "shell", "getprop", "ro.product.model"]
    proc = subprocess.Popen(q, stdin=subprocess.PIPE, stdout=subprocess.PIPE)
    stdout, _ = proc.communicate()
    stdout = stdout.decode()

    return stdout

In [ ]:
def get_ip(device):
    q = ["adb", "-s", device, "shell", "ifconfig", "wlan0"]
    proc = subprocess.Popen(q, stdin=subprocess.PIPE, stdout=subprocess.PIPE)
    stdout, _ = proc.communicate()
    stdout = stdout.decode()

    s = re.findall(r"inet addr:\b\d+(?:\.\d+){3}\b", stdout)
    if len(s) > 0:
        s = s[0]
        s = s[10:]
    else:
        s = None

    return s

In [ ]:
def activate_tcpip(device):
    global TCPIP_DEVICES
    ip = get_ip(device)
    n = ip.split(".")[-1]
    port = 5555 + int(n)

    q = ["adb", "-s", device, "tcpip", str(port)]
    proc = subprocess.Popen(q, stdin=subprocess.PIPE, stdout=subprocess.PIPE)
    stdout, stderr = proc.communicate()
    print(stdout.decode())

    TCPIP_DEVICES.append(ip)

In [ ]:
def connect_tcpip(ip):
    n = ip.split(".")[-1]
    port = 5555 + int(n)

    q = ["adb", "connect", f"{ip}:{port}"]
    proc = subprocess.Popen(q, stdin=subprocess.PIPE, stdout=subprocess.PIPE)
    stdout, stderr = proc.communicate()
    print(stdout.decode())

In [ ]:
def disconnect_tcpip(ip):
    n = ip.split(".")[-1]
    port = 5555 + int(n)

    q = ["adb", "disconnect", f"{ip}:{port}"]
    proc = subprocess.Popen(q, stdin=subprocess.PIPE, stdout=subprocess.PIPE)
    stdout, stderr = proc.communicate()
    print(stdout.decode())

In [ ]:
def run_scrcpy(
    ip, bit_rate="2M", max_fps=24, crop="1600:900:2017:510", full_screen=False
):
    global SCRCPY_PROC
    n = ip.split(".")[-1]
    port = 5555 + int(n)

    env = os.environ.copy()

    q = ["scrcpy", f"--tcpip={ip}:{port}"]

    if bit_rate is not None:
        q.append(f"--video-bit-rate={bit_rate}")
    if max_fps is not None:
        q.append(f"--max-fps={str(max_fps)}")
    if crop is not None:
        q.append(f"--crop={crop}")
    if full_screen:
        q.append("-f")

    SCRCPY_PROC = subprocess.Popen(q, env=env, shell=False)

In [ ]:
def close_scrcpy():
    SCRCPY_PROC.kill()

In [ ]:
def disable_proximity_sensor(device):
    if get_device_model(device) == "Quest 2\r\n":
        q = [
            "adb",
            "-s",
            device,
            "shell",
            "am",
            "broadcast",
            "-a",
            "com.oculus.vrpowermanager.prox_close",
        ]
        proc = subprocess.Popen(q, stdin=subprocess.PIPE, stdout=subprocess.PIPE)
        stdout, stderr = proc.communicate()
        print(stdout.decode())

In [ ]:
def enable_proximity_sensor(device):
    if get_device_model(device) == "Quest 2\r\n":
        q = [
            "adb",
            "-s",
            device,
            "shell",
            "am",
            "broadcast",
            "-a",
            "com.oculus.vrpowermanager.automation_disable",
        ]
        proc = subprocess.Popen(q, stdin=subprocess.PIPE, stdout=subprocess.PIPE)
        stdout, stderr = proc.communicate()
        print(stdout.decode())

In [ ]:
# devices = get_devices()
# device = devices[0]

# print(get_devices())

# activate_tcpip(device)
# time.sleep(3)

# ip = get_ip(device)
# connect_tcpip(ip)

# print(get_devices())

# # run_scrcpy(ip, crop=None, full_screen=True)
# run_scrcpy(ip)

# time.sleep(5)
# close_scrcpy()

# disconnect_tcpip(ip)

# print(get_devices())

In [ ]:
devices = get_usb_devices()

for device in devices:
    activate_tcpip(device)
    time.sleep(3)
    ip = get_ip(device)
    connect_tcpip(ip)

In [ ]:
devices = get_connected_devices()
print(devices)
device = devices[0]
disable_proximity_sensor(device)
time.sleep(3)

ip = get_ip(device)

if SCRCPY_PROC is not None:
    close_scrcpy()

run_scrcpy(ip)

In [ ]:
devices = get_connected_devices()

device = devices[1]
disable_proximity_sensor(device)

In [ ]:
# List all connected devices, including IP connected devices

# q = ["adb", "devices"]
# proc = subprocess.Popen(q, stdin=subprocess.PIPE, stdout=subprocess.PIPE)
# stdout, stderr = proc.communicate()
# print(stdout.decode())

In [ ]:
# # Turn on TCPIP mode for a given device

# q = ["adb", "tcpip", "5555"]
# proc = subprocess.Popen(q, stdin=subprocess.PIPE, stdout=subprocess.PIPE)
# stdout, stderr = proc.communicate()
# print(stdout.decode())

In [ ]:
# # If only one device is connected, get its IP address
# # If more than one device is connected, use `adb -s <device> shell ifconfig wlan0`

# q = ["adb", "shell", "ifconfig", "wlan0"]
# proc = subprocess.Popen(q, stdin=subprocess.PIPE, stdout=subprocess.PIPE)
# stdout, stderr = proc.communicate()
# print(stdout.decode())

In [ ]:
# # Connect to the TCPIP device. It should now appear in `adb devices`

# q = ["adb", "connect", "192.168.137.77:5555"]
# proc = subprocess.Popen(q, stdin=subprocess.PIPE, stdout=subprocess.PIPE)
# stdout, stderr = proc.communicate()
# print(stdout.decode())

In [ ]:
# Oculus only. Disable/enable the proximity sensor

# q = ["adb", "shell", "am", "broadcast", "-a", "com.oculus.vrpowermanager.automation_disable"]
# # q = ["adb", "shell", "am", "broadcast", "-a", "com.oculus.vrpowermanager.prox_close"]
# proc = subprocess.Popen(q, stdin=subprocess.PIPE, stdout=subprocess.PIPE)
# stdout, stderr = proc.communicate()
# print(stdout.decode())

In [ ]:
# Run scrcpy

# env = os.environ.copy()
# q = ["scrcpy"]
# proc = subprocess.Popen(q, env=env, shell=True, stdin=subprocess.PIPE, stdout=subprocess.PIPE)
# stdout, stderr = proc.communicate()
# print(stdout.decode())